### 1.查找文件中的中文字符

In [2]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

In [5]:
OSMFILE = 'shanghai.osm'
chinese_end = re.compile(u"[\u4e00-\u9fa5]$")

def audit(osmfile):
    osm_file = open(OSMFILE,'r')
    street_types = defaultdict(set)
    for event,ele in ET.iterparse(osm_file,events=('start',)):
        for tag in ele.iter('tag'):
            if chinese_end.search(tag.attrib['v']):
                print tag.attrib['v']

if __name__=='__main__':
    audit(OSMFILE)

莘庄
莘庄
步云
步云
中山北二路/大柏树
开往大柏树, 用曲阳路
中山北二路/大柏树
开往大柏树, 用曲阳路
延安高架路
延安高架路
比利时领事馆
比利时领事馆
内环高架路
内环高架路
五洲大道
五洲大道
军工路
军工路
逸仙高架路/绕城高速
逸仙高架路/绕城高速
南北高架路 卢浦大桥
南北高架路 卢浦大桥
南北高架路 共和新路立交方向
南北高架路 共和新路立交方向
卢浦大桥 S20 浦东机场
卢浦大桥 S20 浦东机场
杨高南路 林海公路
杨高南路 林海公路
龙华西路
龙华西路
桂林路 上海南站
桂林路 上海南站
瑞金南路
瑞金南路
张浦
张浦
沪南公路
沪南公路
徐家汇路
徐家汇路
淮海中路
淮海中路
沪金高速颛桥收费站
沪金高速颛桥收费站
中山东一路汉口路
中山东一路汉口路
江西中路
江西中路
南北高架路
南北高架路
G50 湖州方向 延安高架路
G50 湖州方向 延安高架路
申江路
申江路
中山北一路/逸仙路/中环路/翔殷路隧道
中山北一路/逸仙路/中环路/翔殷路隧道
S5 嘉定城区方向
S5 嘉定城区方向
铜川路
铜川路
G2 G42 苏州方向
G2 G42 苏州方向
金沙江路
金沙江路
宝山区
宝山区
宝山区
宝山区
长宁区
长宁区
长宁区
长宁区
崇明县
崇明县
崇明县
崇明县
青村镇
青村镇
青村镇
青村镇
海门市
海門市
海门市
海门市
海門市
海门市
虹口区
虹口区
虹口区
虹口区
黄浦区
黄浦区
黄浦区
黄浦区
嘉善县
嘉善县
嘉善县
嘉善县
嘉定区
嘉定区
嘉定区
嘉定区
金山区
金山区
金山区
金山区
静安区
静安区
静安区
静安区
嘉兴市
嘉興市
嘉兴市
嘉兴市
嘉興市
嘉兴市
平湖市
平湖市
平湖市
平湖市
平湖市
平湖市
青浦区
青浦区
启东市
啓東市
启东市
启东市
啓東市
启东市
普陀区
普陀区
普陀区
普陀区
上海市
上海市
上海市
沪
上海市
上海市
上海市
沪
太仓市
太倉市
太仓市
太仓市
太倉市
太仓市
松江区
松江区
松江区
松江区
徐汇区
徐汇区
徐汇区
徐汇区
五角场街道
上海
五角场街道
上海
浏河镇
浏河镇
浏河镇
浏河镇
浮桥镇
浮桥镇
浮桥镇
浮桥镇
陆家镇
陆家镇
陆家镇
陆家镇
巴城镇
巴城镇
巴城镇
巴城镇
周市镇
周市镇
周市镇
周市镇
淀山湖镇
淀山湖镇
淀山湖镇
淀山湖镇
胜浦街道
胜浦镇
胜

### 2.删除异常字符

In [ ]:
def update(k,v):
    if '=&#' in v:
        return v.replace('=&#','')
    

In [ ]:
### 3.将数据写入到csv文件中

In [27]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus

import schema

In [28]:
OSM_PATH = "shanghai.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

In [29]:
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_tag(el,tag):
    v = update(tag.attrib['k'],tag.attrib['v'])
    tag={
        'id':el.attrib['id'],
        'key':tag.attrib['k'],
        'value':v,
        'type':'regular'
    }
    if LOWER_COLON.match(tag['key']):
        tag['type'], _, tag['key'] = tag['key'].partition(':')
    return tag

def shape_way_node(el,i,nd):
    return {
        'id':el.attrib['id'],
        'node_id':nd.attrib['ref'],
        'position':i
    }

def shape_element(el, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    tags = [shape_tag(el,t) for t in el.iter('tag')]
    if el.tag == 'node':
        node_attribs = {f: el.attrib[f] for f in node_attr_fields}
        
        return {'node': node_attribs, 'node_tags': tags}
    elif el.tag == 'way':
        way_attribs = {f: el.attrib[f] for f in way_attr_fields}
        
        way_nodes = [shape_way_node(el,i,nd)
                    for i,nd in enumerate(el.iter('nd'))]
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    """如果是正确的类型时,返回标签中的tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    """当和schema的数据格式不匹配时,抛出异常"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""
    """扩展csv下的DictWriter方法的去支持Unicode输入"""
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
    """将处理好的xml文件写入到csv中"""
    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'wb') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
    

In [30]:
if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)